# Scraping Houses in Redfin

**To be completed INDIVIDUALLY and due on April 10 at 7pm.**

In this homework, we will practice web scraping. Let's get some basic information for each house/apartment in Massachusetts.

You can see houses around Boston University [here](https://www.redfin.com/zipcode/02215)

![Information to be scraped](redfin.png)


On each house page, scrape the information of this [house](https://www.redfin.com/MA/Boston/120-Mountfort-St-02215/unit-502/home/11838719), including 
* full address
* price
* number of beds
* number of baths (full or not)
* Sq. Ft (square feet)
* price per Sq. Ft
* property type
* County
* community
* built
* Property details (e.g. dishwasher, elevator, parking, heating, air conditioning, maintenance, etc.)

Try to extract as many features as possible. You will use this information in future homework. 


Save the data in "house.csv" in the following format:

full address, price, beds, baths, ......

**To receive credit, you must commit house.csv to Github.** **(20 pts)**

In [1]:
# Aaron Elliot
# ellioa2@bu.edu
# U71617701
# HW 4.1 4/10/17
from selenium import webdriver
from pyvirtualdisplay import Display
import time
import random
import csv
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
display = Display(visible=0, size=(800, 600))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '800x600x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '800x600x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [4]:
def scrape_RedfinZipCode(zip):
    url = 'https://www.redfin.com/zipcode/'+zip
    display = Display(visible=0, size=(800, 600))
    display.start()
    
    time.sleep(2+3*random.random())
    driver = webdriver.Chrome()
    driver.get(url)
    
    
    #return(driver.current_url)

    if driver.current_url == 'https://www.redfin.com/out-of-area-signup':
        driver.delete_all_cookies()
        return(None)
    if driver.find_element_by_class_name('searchAboveTheFold')\
    .find_element_by_xpath(".//div[@data-rf-test-id='homes-description']").text == '0 Homes':
        driver.delete_all_cookies()
        return(None)
    
    Table = driver.find_element_by_xpath("//span[@data-rf-test-name='tableOption']")
    Table.click()
    del(Table)

    #Webpages = driver.find_element_by_xpath("//tr[class='tableRow']")
    Rows = driver.find_elements_by_class_name('tableRow')
    WebPages = []
    for Row in Rows:
        #child = Page.find_elements_by_css_selector("*/input[2]")
        child = Row.find_element_by_class_name('col_address')\
                .find_element_by_xpath(".//a")
        WebPages.append(child.get_attribute('href'))
    del(Rows)

    DicList= []
    
    if (random.random()>.9):
        time.sleep(3+2*random.random())
        driver.get('https://www.redfin.com')
    
    for Page in WebPages:
        housedic = {}
        time.sleep(3+4*random.random()) # add some delay to mimic dumb human
        driver.get(Page)
        MainStats = driver.find_element_by_class_name("main-stats")
        
        try:
            housedic['street_address'] = MainStats.find_element_by_class_name('street-address').get_attribute('title')
        except NoSuchElementException:
            X=1
        try:
            housedic['price'] = MainStats.find_element_by_xpath(".//span[@itemprop='price']").get_attribute('content')\
                            +MainStats.find_element_by_xpath(".//span[@itemprop='priceCurrency']").get_attribute('content')
        except NoSuchElementException:
            X=1
        try:
            housedic['beds'] = driver.find_element_by_xpath("//div[@data-rf-test-id='abp-beds']")\
                .find_element_by_class_name("statsValue").text
        except NoSuchElementException:
            X=1
        try:
            housedic['baths'] = driver.find_element_by_xpath("//div[@data-rf-test-id='abp-baths']")\
            .find_element_by_class_name("statsValue").text
        except NoSuchElementException:
            X=1
        try:
            SqFt = driver.find_element_by_xpath("//div[@data-rf-test-id='abp-sqFt']")
            housedic['sqft'] = SqFt.find_element_by_class_name("statsValue").text
            housedic['price/sqft'] = SqFt.find_element_by_class_name("statsLabel").text
        except NoSuchElementException:
            X=1
        KeyDetails = driver.find_element_by_class_name("keyDetailsList")
        
        
        
        try:
            housedic['prop type'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'Property Type')]/following-sibling::*").text
        except NoSuchElementException:
            X=1
        try:
            housedic['county'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'County')]/following-sibling::*").text
        except NoSuchElementException:
            X=1
        try:
            housedic['built'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'Built')]/following-sibling::*").text
        except NoSuchElementException:
            X=1
        try:
            housedic['community'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'Community')]/following-sibling::*").text
        except NoSuchElementException:
            X=1
            
        try:
            details = driver.find_element_by_class_name("remarks").find_element_by_xpath(".//p").text
            details =  ''.join(filter(lambda l: l.isalpha() or l==' ', details.lower().replace('-',' ')))

            if ' parking' in details or ' spot to park' in details:
                housedic['parking']=True
            if ' air conditioning' in details or ' air conditioned' in details:
                housedic['air conditioning']=True    
            if 'kitchen' in details or 'cutty cooking room' in details:
                housedic['kitchen']=True

            details = set(details.split(' '))    
            if 'dishwasher' in details or 'dishwashing' in details:
                housedic['dishwasher']=True
            if 'laundry' in details:
                housedic['laundry']=True
            if 'heating' in details or 'heated' in details:
                housedic['heating']=True
            if 'maintenance ' in details:
                housedic['maintenance']=True
            if 'elevator' in details:
                housedic['elevator']=True
            if 'stairs' in details:
                housedic['stairs']=True
            if 'refrigerator' in details or 'refridgerator' in details:
                housedic['refrigerator']=True
            if 'furnished' in details or 'furnishings' in details:
                housedic['furnished']=True
        except NoSuchElementException:
            X=1
        DicList.append(housedic)
        if (random.random()>.99):
            time.sleep(5+6*random.random())
            driver.get('https://www.redfin.com')


    driver.delete_all_cookies()
    return (DicList)

In [4]:
print("Starting")
zip = '02215'
DicList = scrape_RedfinZipCode(zip)
print(DicList)
print("Finished")

Starting
[{'street_address': '86 Jersey St #42', 'price': '474900USD', 'beds': '1', 'baths': '1', 'sqft': '538', 'price/sqft': '$883 / Sq. Ft.', 'prop type': 'Condominium', 'county': 'Suffolk', 'built': '1920', 'community': 'The Fenway', 'kitchen': True, 'laundry': True}, {'street_address': '24-30 Peterborough St #23', 'price': '429000USD', 'beds': '1', 'baths': '1', 'sqft': '476', 'price/sqft': '$901 / Sq. Ft.', 'prop type': 'Condominium', 'county': 'Suffolk', 'built': '1900', 'community': 'The Fenway', 'kitchen': True}, {'street_address': '64 Queensberry St #312', 'price': '369000USD', 'beds': '1', 'baths': '1', 'sqft': '360', 'price/sqft': '$1025 / Sq. Ft.', 'prop type': 'Condominium', 'county': 'Suffolk', 'built': '1910', 'community': 'Boston', 'kitchen': True, 'dishwasher': True}, {'street_address': '849 Beacon #2', 'price': '549000USD', 'beds': '1', 'baths': '1', 'sqft': '550', 'price/sqft': '$998 / Sq. Ft.', 'prop type': 'Condominium', 'county': 'Suffolk', 'built': '1920', 'comm

In [5]:
print("writing begins")

keys = ['street_address','price','beds','baths','sqft','price/sqft',\
    'prop type','county','community','built','parking','air conditioning',\
    'kitchen','dishwasher','laundry','heating','maintenance','elevator',\
    'stairs','refrigerator','furnished']

with open('house.csv', 'w') as output_file:
    csvWriter = csv.DictWriter(output_file, keys)
    csvWriter.writeheader()
    csvWriter.writerows(DicList)
#del (DicList)
print("writing finished")   

writing begins
writing finished


-------

Now that you can scrape and extract information of a house, scrape the information of houses in Massachusetts (as of today there are around 10000 houses in MA listed in Redfin). To see all the listings, you can query all the zip codes in Massachusetts. You can see all the zip codes in Massachusetts [here](https://github.com/evimaria/CS506-Spring2007/blob/master/Homeworks/zipcodes.txt).
Save the data in "MA_houses.csv", one line for each house:
**To receive credit, you must commit MA_houses.csv to Github** **(20 pts)**

In [1]:
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# X NOT THE FINAL METHOD USED, SCROLL DOWN TO BOTTOM FOR FINAL METHOD X
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
print("Starting Method 1: finding the CSV's urls")
#this is not the final method I used please look further for my final method
#I was quickly banned and the method does not recognize this
# This code was used to construct MA_houses2.csv which I did not commit to the final file

from urllib.parse import urljoin  # for Python2: from urlparse import urljoin
from urllib.request import urlretrieve
from urllib.request import urlopen
import os

from selenium import webdriver
from pyvirtualdisplay import Display
import time
import random
import csv
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
display = Display(visible=0, size=(800, 600))
display.start()

display = Display(visible=0, size=(800, 600))
display.start()
def get_csv_RedfinZipCode(zip,driver):
    url = 'https://www.redfin.com/zipcode/'+zip
     
    wait = WebDriverWait(driver, 10)
    time.sleep(random.random())
    driver.get(url)
    
    if driver.current_url != url:
        driver.delete_all_cookies()
        return(None)
    
    try: csv_url = urljoin(url, driver.find_element_by_class_name('searchAboveTheFold').\
                      find_element_by_class_name('viewingPage').find_element_by_xpath(".//a")\
                      .get_attribute("href"))
    except NoSuchElementException:
        return (None)
    return (csv_url)

Starting Method 1: finding the CSV's urls


In [6]:
print("Method 1: Check each Zipcode for a CSV and use get_csv_RedfinZipCode to append the address of the csv to csv_urls.txt")

driver = webdriver.Chrome()
i=0

Empty_Zips = 0
Number_of_csv = 0
# this one doesnt work
with open('csv_urls.txt','w') as csv:
    for zip_code in open('zipcodes.txt','r'): 
        if i>0:
            url = get_csv_RedfinZipCode(zip_code.strip(),driver)
            if url==None:
                Empty_Zips = Empty_Zips + 1
                i=i+1
                continue
            csv.write(str(url))
            csv.write('\n')
            Number_of_csv = Number_of_csv + 1
        i=i+1

Method 1: Check each Zipcode for a CSV and use get_csv_RedfinZipCode to append the address of the csv to csv_urls.txt


In [8]:
print("Method 1: Request the CSV's w/user_agent and append the data to MA_houses2")
import requests
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.76 Safari/537.36"
keys = ['street_address','price','beds','baths','sqft','price/sqft',\
    'prop type','county','community','built','parking','air conditioning',\
    'kitchen','dishwasher','laundry','heating','maintenance','elevator',\
    'stairs','refrigerator','furnished']
house_count = 0

with open('MA_houses2.csv', 'a') as output_file:
    for url in open('csv_urls.txt', 'r'):
        r = requests.get(url.strip(), stream=True,headers={'User-Agent': user_agent})
        time.sleep(3+random.random()*2)
        DicList = []
        with open('Failed_zips.csv', 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        with open('Failed_zips.csv','r') as f:
            reader = csv.reader(f)
            for home in reader:
                house_count = house_count + 1
                Dic = {}
                Dic['street_address'] = home[3]
                Dic['price'] = home[7]
                Dic['beds'] = home[8]
                Dic['baths'] = home[9]
                Dic['sqft'] = home[11]
                Dic['price/sqft'] = home[15]
                Dic['prop type'] = home[2]
                Dic['built'] = home[13]
                DicList.append(Dic)
    csvWriter = csv.DictWriter(output_file, keys)
    csvWriter.writeheader()
    csvWriter.writerows(DicList)
                
print('finished')

Method 1: Request the CSV's w/user_agent and append the data to MA_houses2
finished


In [1]:
# testing space
i=-1
for zip_code in open('zipcodes.txt','r'):
    i=i+1
    if i>485:
        print(i, zip_code)
    Lastzip = zip_code
print(i,Lastzip)


486 02207

487 02210

488 02211

489 02212

490 02215

491 02216

492 02217

493 02222

494 02228

495 02238

496 02239

497 02241

498 02266

499 02269

500 02283

501 02284

502 02293

503 02295

504 02297

505 02298

506 02301

507 02302

508 02303

509 02304

510 02305

511 02322

512 02324

513 02325

514 02327

515 02330

516 02331

517 02332

518 02333

519 02334

520 02337

521 02338

522 02339

523 02340

524 02341

525 02343

526 02344

527 02345

528 02346

529 02347

530 02348

531 02349

532 02350

533 02351

534 02355

535 02356

536 02357

537 02358

538 02359

539 02360

540 02361

541 02362

542 02364

543 02366

544 02367

545 02368

546 02370

547 02375

548 02379

549 02381

550 02382

551 02420

552 02421

553 02445

554 02446

555 02447

556 02451

557 02452

558 02453

559 02454

560 02455

561 02456

562 02457

563 02458

564 02459

565 02460

566 02461

567 02462

568 02464

569 02465

570 02466

571 02467

572 02468

573 02471

574 02472

575 02474

576 02475


In [1]:
# testing space

In [ ]:
# testing space

In [ ]:
# testing space

In [3]:
# Method2: Selenium webdriver, webscraping
# This is my best method, it was placed in MAhouses.py and so was not ran from this console
# This is the code used to construct MA_houses.csv
# At a certain point, their security was able to recognize my script regardless of what tweaks I did,
# so I was only able to reach the 
#!/usr/bin/python

from selenium import webdriver
from pyvirtualdisplay import Display
import time
import random
import csv
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome()
driver.implicitly_wait(10)

def scrape_RedfinZipCode(zip,driver):
    url = 'https://www.redfin.com/zipcode/'+zip
    
    
    wait = WebDriverWait(driver, 10)
    
    
    if (random.random()>.94):
        driver.get('https://www.redfin.com')
        time.sleep(random.random()+3)
     

    driver.get(url)
    time.sleep(random.random()*2+3)

    
    # if they redirected me, then skip it
    if driver.current_url != url:
        driver.delete_all_cookies()
        return(None)
    
    # if the zip has 0 homes in it, then skip it
    if driver.find_element_by_class_name('searchAboveTheFold')\
        .find_element_by_xpath(".//div[@data-rf-test-id='homes-description']")\
        .text == '0 Homes':
        driver.delete_all_cookies()
        return(None)
    
    Table = driver.find_element_by_xpath("//span[@data-rf-test-name='tableOption']")
    Table.click()
    del(Table)
    
    # this adds the links of the houses in ZIP to Webpages
    Rows = driver.find_elements_by_class_name('tableRow')
    WebPages = [] 
    for Row in Rows:
        child = Row.find_element_by_class_name('col_address')\
                .find_element_by_xpath(".//a")
        WebPages.append(child.get_attribute('href'))
    del(Rows)

    HouseList = [] # this is our main data structure to append house dictionaries
    time.sleep(.5)
    # this looks at each house's link and gets the information on it
    for Page in WebPages:
        housedic = {}
        #if (random.random()>.97):
        #    driver.get('https://www.redfin.com')
        #    time.sleep(random.random()+11)
        driver.get(Page)
        time.sleep(random.random()*3+1)
        MainStats = driver.find_element_by_class_name("main-stats")
        
        try:
            housedic['street_address'] = MainStats.find_element_by_class_name('street-address').get_attribute('title')
        except NoSuchElementException:
            pass
        
        try:
            housedic['price'] = MainStats.find_element_by_xpath(".//span[@itemprop='price']").get_attribute('content')
        except NoSuchElementException:
            pass
        
        try:
            housedic['beds'] = driver.find_element_by_xpath("//div[@data-rf-test-id='abp-beds']")\
                .find_element_by_class_name("statsValue").text
        except NoSuchElementException:
            pass
        
        try:
            housedic['baths'] = driver.find_element_by_xpath("//div[@data-rf-test-id='abp-baths']")\
            .find_element_by_class_name("statsValue").text
        except NoSuchElementException:
            pass
        
        try:
            SqFt = driver.find_element_by_xpath("//div[@data-rf-test-id='abp-sqFt']")
            housedic['sqft'] = SqFt.find_element_by_class_name("statsValue").text
            housedic['price/sqft'] = SqFt.find_element_by_class_name("statsLabel").text
        except NoSuchElementException:
            pass
        
        KeyDetails = driver.find_element_by_class_name("keyDetailsList")

        try:
            housedic['prop type'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'Property Type')]/following-sibling::*").text
        except NoSuchElementException:
            pass
        try:
            housedic['county'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'County')]/following-sibling::*").text
        except NoSuchElementException:
            pass
        try:
            housedic['built'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'Built')]/following-sibling::*").text
        except NoSuchElementException:
            pass
        try:
            housedic['community'] = KeyDetails.find_element_by_xpath(".//*[contains(text(), 'Community')]/following-sibling::*").text
        except NoSuchElementException:
            pass
            
        try:
            details = driver.find_element_by_class_name("remarks").find_element_by_xpath(".//p").text
            details =  ''.join(filter(lambda l: l.isalpha() or l==' ', details.lower().replace('-',' ')))

            if ' parking' in details or ' spot to park' in details:
                housedic['parking']=True
            if ' air conditioning' in details or ' air conditioned' in details:
                housedic['air conditioning']=True    

            details = set(details.split(' '))
            if 'kitchen' in details:
                housedic['kitchen'] = True
            if 'dishwasher' in details or 'dishwashing' in details:
                housedic['dishwasher'] = True
            if 'laundry' in details:
                housedic['laundry'] = True
            if 'heating' in details or 'heated' in details:
                housedic['heating'] = True
            if 'elevator' in details:
                housedic['elevator'] = True
        except NoSuchElementException:
            pass
        
        HouseList.append(housedic)
    driver.delete_all_cookies()
    return (HouseList)

print("Starting")

keys = ['street_address','price','beds','baths','sqft','price/sqft',\
    'prop type','county','community','built','parking','air conditioning',\
    'kitchen','dishwasher','laundry','heating','elevator']


i=-1
count_found = 0
count_failed = 0

with open('MA_houses.csv', 'a') as output_file:
    MainCSV = csv.DictWriter(output_file, keys)
    for zipcode in open('zipcodes.txt','r'):
        i+=1
        if i>512: # Whenever their security locked me out, I would manually update this value and reset my IP
                  # I usually got around 50 - 200 Zips between resets, but toward the deadline their security
                  # suddenly started recognizing me sooner, and so I was only getting around 10 - 20 zips.
                    # The 512th Zip was as far as I was able to get. Also I ended up with the zips that I
                    # accidentally ran twice, so there are a small number of duplicates in my MA_houses.csv
            if (i%98 == 0):
                time.sleep(60*3+3 +random.random())
                print("Coffee break!")
            print(i,zipcode.strip())
            DicList = scrape_RedfinZipCode(zipcode.strip(),driver)
            if DicList != None:
                count_found = count_found+1
                MainCSV.writerows(DicList)
            else:
                count_failed = count_failed+1
    
print("finished\n","count failed:",count_failed,"\ncount found:",count_found)

1027


In [2]:
# these were the csv url I printed as output before I reorginized the file,
# I didnt think it was worth wasting time re-running to have it under the correct code,
# but still interesting

https://www.redfin.com/stingray/api/gis-csv?al=1&market=boston&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=199&region_type=2&sf=1,2,3,5,6,7&sp=true&status=9&uipt=1,2,3,4,5,6&v=8&zoomLevel=11

https://www.redfin.com/stingray/api/gis-csv?al=1&market=boston&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=216&region_type=2&sf=1,2,3,5,6,7&sp=true&status=9&uipt=1,2,3,4,5,6&v=8&zoomLevel=12

https://www.redfin.com/stingray/api/gis-csv?al=1&market=boston&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=222&region_type=2&sf=1,2,3,5,6,7&sp=true&status=9&uipt=1,2,3,4,5,6&v=8&zoomLevel=12

https://www.redfin.com/stingray/api/gis-csv?al=1&market=boston&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=238&region_type=2&sf=1,2,3,5,6,7&sp=true&status=9&uipt=1,2,3,4,5,6&v=8&zoomLevel=11

https://www.redfin.com/stingray/api/gis-csv?al=1&market=boston&num_homes=350&ord=redfin-recommended-asc&page_number=1&region_id=239&region_type=2&sf

# Note that many websites will block you if you are sending too many queries. To prevent getting block, you should wait between queries (typically 2-5 seconds). 

-------